In [1]:
!pip install transformers torch datasets evaluate


[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch
import torch.nn as nn
from transformers import (
    BartModel,
    BartTokenizer,
    BertModel,
    BertTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
)
from datasets import load_dataset

In [4]:
dataset = load_dataset("cnn_dailymail", "3.0.0")
train_data = dataset["train"]
validation_data = dataset["validation"]

# Step 3: Load Tokenizers
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


# Set device
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Preprocessing Function
def preprocess_function(examples):
    bart_inputs = bart_tokenizer(examples["article"], max_length=1024, truncation=True, padding="max_length")
    labels = bart_tokenizer(examples["highlights"], max_length=128, truncation=True, padding="max_length")["input_ids"]

    return {
        "input_ids": torch.tensor(bart_inputs["input_ids"]).to(device),
        "attention_mask": torch.tensor(bart_inputs["attention_mask"]).to(device),
        "labels": torch.tensor(labels).to(device),
    }

# Apply preprocessing
tokenized_train_data = train_data.map(preprocess_function, batched=True)
tokenized_validation_data = validation_data.map(preprocess_function, batched=True)

# Custom Data Collator
class CustomDataCollator(DataCollatorForSeq2Seq):
    def __call__(self, batch):
        batch = super().__call__(batch)
        return {key: value.to(device) if isinstance(value, torch.Tensor) else value for key, value in batch.items()}

data_collator = CustomDataCollator(tokenizer=bart_tokenizer, model=dual_encoder_model)

# Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    fp16=True,  # Enable mixed precision
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    predict_with_generate=True,
)

# Trainer
trainer = Seq2SeqTrainer(
    model=dual_encoder_model.to(device),
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_validation_data,
    tokenizer=bart_tokenizer,
    data_collator=data_collator,
)

# Train
trainer.train()


Using device: mps


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

NameError: name 'dual_encoder_model' is not defined